In [ ]:
# This program is using all the previuosly trained models
# to give the final result.
# first it will identify which part of the upper limb is
# in the image.  Then it uses the corresponding model to 
# predict the abnormality.

In [1]:
import os
import numpy as np
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation,Dropout
from keras import regularizers
from keras.optimizers import Adam, Adadelta
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
from keras.models import model_from_json
from IPython.display import Image
%matplotlib inline


Using TensorFlow backend.


In [2]:
# Set the identification dictionary
classification_dic={0:'ELBOW', 1:'FINGER', 2:'FOREARM',3:'HAND',4:'HUMERUS',5:'SHOULDER',6:'WRIST'}
abnormality_dic={0:'negative', 1:'positive'}


In [3]:
# Load the pretrained models

#(1)
# Upper Limb Classification model aschitecture
print('Loadind ULC model...')
with open('h5/model_architecture_ULCwAnshu.json', 'r') as f:
    ULC_model = model_from_json(f.read())
# Upper Limb Classification model weights
ULC_model.load_weights('h5/modelMN_ULC_best03w Anshu.h5')

#(2)
print('Loadind Elbow model...')
# Elbow Abnormality detection model aschitecture
with open('h5/model_architectureXR_ELBOW final.json', 'r') as f:
    elbow_model = model_from_json(f.read())
# Elbow Abnormality detection model weights
elbow_model.load_weights('h5/modelMNXR_ELBOW final.h5')

#(3)
print('Loadind Finger model...')
# Finger Abnormality detection model aschitecture
with open('h5/model_architectureXR_FINGER_final.json', 'r') as f:
    finger_model = model_from_json(f.read())
# Finger Abnormality detection model weights
finger_model.load_weights('h5/modelMNXR_FINGER_final.h5')

#(4)
print('Loadind Forearm model...')
# Forearm Abnormality detection model aschitecture
with open('h5/model_architectureXR_FOREARM_final.json', 'r') as f:
    forearm_model = model_from_json(f.read())
# Forearm Abnormality detection model weights
forearm_model.load_weights('h5/modelMNXR_FOREARM_final.h5')

#(5)
print('Loadind Hand model...')
# Hand Abnormality detection model aschitecture
with open('h5/model_architectureXR_HAND_final1.json', 'r') as f:
    hand_model = model_from_json(f.read())
# Hand Abnormality detection model weights
hand_model.load_weights('h5/modelMNXR_HAND_final1.h5')

#(6)
print('Loadind Humerus model...')
# Humerus Abnormality detection model aschitecture
with open('h5/model_architectureXR_HUMERUS_final.json', 'r') as f:
    humerus_model = model_from_json(f.read())
# Humerus Abnormality detection model weights
humerus_model.load_weights('h5/modelMNXR_HUMERUS_final.h5')

#(7)
print('Loadind Shoulder model...')
# Shoulder Abnormality detection model aschitecture
with open('h5/model_architectureXR_SHOULDERfinal2.json', 'r') as f:
    shoulder_model = model_from_json(f.read())
# Shoulder Abnormality detection model weights
shoulder_model.load_weights('h5/modelMNXR_SHOULDERfinal2.h5')

#(8)
print('Loadind Wrist model...')
# Wrist Abnormality detection model aschitecture
with open('h5/model_architecture_XR_WRIST_final.json', 'r') as f:
    wrist_model = model_from_json(f.read())
# Wrist Abnormality detection model weights
wrist_model.load_weights('h5/model_MNXR_WRIST_final.h5')

print('All models are downloaded!!')

Loadind ULC model...
Instructions for updating:
Colocations handled automatically by placer.
Loadind Elbow model...
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loadind Finger model...
Loadind Forearm model...
Loadind Hand model...
Loadind Humerus model...
Loadind Shoulder model...
Loadind Wrist model...
All models are downloaded!!


In [4]:
# prepare the image for model prediction
def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [5]:
# Prediction functions
def ulc_predict(image):
    preprocessed_image = prepare_image(image)
    predictions = ULC_model.predict(preprocessed_image)
    return classification_dic[np.argmax(predictions, axis=1)[0]]
    
def elbow_predict(image):
    preprocessed_image = prepare_image(image)
    predictions = elbow_model.predict(preprocessed_image)
    return abnormality_dic[np.argmax(predictions, axis=1)[0]],predictions[0][1]

def finger_predict(image):
    preprocessed_image = prepare_image(image)
    predictions = finger_model.predict(preprocessed_image)
    return abnormality_dic[np.argmax(predictions, axis=1)[0]],predictions[0][1]

def forearm_predict(image):
    preprocessed_image = prepare_image(image)
    predictions = forearm_model.predict(preprocessed_image)
    return abnormality_dic[np.argmax(predictions, axis=1)[0]],predictions[0][1]

def hand_predict(image):
    preprocessed_image = prepare_image(image)
    predictions = hand_model.predict(preprocessed_image)
    return abnormality_dic[np.argmax(predictions, axis=1)[0]],predictions[0][1]

def humerus_predict(image):
    preprocessed_image = prepare_image(image)
    predictions = humerus_model.predict(preprocessed_image)
    return abnormality_dic[np.argmax(predictions, axis=1)[0]],predictions[0][1]

def shoulder_predict(image):
    preprocessed_image = prepare_image(image)
    predictions = shoulder_model.predict(preprocessed_image)
    return abnormality_dic[np.argmax(predictions, axis=1)[0]],predictions[0][1]

def wrist_predict(image):
    preprocessed_image = prepare_image(image)
    predictions = wrist_model.predict(preprocessed_image)
    return abnormality_dic[np.argmax(predictions, axis=1)[0]],predictions[0][1]

In [6]:
# Abnormality model selection: after part classifier identify the part
# of the upper limb, it directs the image to the corresponding model 
# for abnormality detection
def abnormality(part,imfile):
    if part == 'ELBOW': 
        return elbow_predict(imfile)
    if part == 'FINGER': 
        return finger_predict(imfile)
    if part == 'FOREARM': 
        return forearm_predict(imfile)
    if part == 'HAND': 
        return hand_predict(imfile)
    if part == 'HUMERUS': 
        return humerus_predict(imfile)
    if part == 'SHOULDER': 
        return shoulder_predict(imfile)
    if part == 'WRIST': 
        return wrist_predict(imfile)
 

In [7]:
# Apply the test on the MURA Dataset
def model_test(dataset):
    study_overall=[]
    image_overall=[]
    for xr in ['XR_ELBOW','XR_FINGER','XR_FOREARM','XR_HAND','XR_HUMERUS','XR_SHOULDER','XR_WRIST']:
        img_count=0
        img_pos=0
        img_neg=0
        img_fpos=0
        img_fneg=0
        true_pos=0
        true_neg=0
        false_pos=0
        false_neg=0
        false_pos_lst=[]
        false_neg_lst=[]
        templst=[]
        path='MURA-v1.1/'+dataset+'/'+xr+'/'
        print('')
        print('')
        print('Results for ',xr)
        for patient in os.listdir(path):
            for study in os.listdir(path+patient):
                total_pred=0
                for img in os.listdir(path+patient+'/'+study):
                    img_count +=1
                    #print(path+patient+'/'+study+'/'+img)
                    pred, pred_prob=abnormality(ulc_predict(path+patient+'/'+study+'/'+img),path+patient+'/'+study+'/'+img)
                    if pred=='positive' and study.split('_')[1] == 'positive':
                        img_pos += 1
                    elif pred=='negative' and study.split('_')[1] == 'negative':
                        img_neg += 1
                    elif pred=='positive' and study.split('_')[1] != 'positive':
                        img_fpos += 1
                    else:
                        img_fneg += 1

                    total_pred += pred_prob
                avg_pred = total_pred/len(os.listdir(path+patient+'/'+study))
                if avg_pred <= 0.5:
                    abnorm_pred='negative'
                else:
                    abnorm_pred='positive'

                if abnorm_pred == study.split('_')[1] == 'positive':
                    true_pos +=1
                elif abnorm_pred == study.split('_')[1] == 'negative':
                    true_neg +=1
                elif abnorm_pred != study.split('_')[1] == 'negative':
                    false_pos +=1
                    false_pos_lst.append(patient+'/'+study)
                else:
                    false_neg +=1
                    false_neg_lst.append(patient+'/'+study)

        ## Testing on the original MURA Dataset (Before doing)
        print('Total number of images = ',img_count)
        print('True Positive = ',img_pos)
        print('True Nigative= ',img_neg)
        print('False Positive = ',img_fpos)
        print('False Nigative= ',img_fneg)
        img_acc=100*(img_pos+img_neg)/(img_pos+img_neg+img_fpos+img_fneg)
        print('Accuracy on IMAGE level =', img_acc)
        image_overall.append(img_acc)    
        print('')
        print('True Positive = ',true_pos)
        print('True Nigative= ',true_neg)
        print('False Positive = ',false_pos)
        print('False Nigative= ',false_neg)
        std_acc=100*(true_neg+true_pos)/(true_neg+true_pos+false_neg+false_pos)
        print('Accuracy on STUDY level =', std_acc)  
        study_overall.append(std_acc)
    print('')
    print('')
    print('Overall Accuracy on IMAGE level= ', sum(image_overall)/len(image_overall))
    print('')
    print('Overall Accuracy on STUDY level= ', sum(study_overall)/len(study_overall))

In [8]:
# output the main model performance againest the valid set provided 
# by Stanford University in MURA dataset
model_test('valid')



Results for  XR_ELBOW
Total number of images =  465
True Positive =  158
True Nigative=  227
False Positive =  8
False Nigative=  72
Accuracy on IMAGE level = 82.79569892473118

True Positive =  49
True Nigative=  89
False Positive =  3
False Nigative=  17
Accuracy on STUDY level = 87.34177215189874


Results for  XR_FINGER
Total number of images =  461
True Positive =  169
True Nigative=  167
False Positive =  47
False Nigative=  78
Accuracy on IMAGE level = 72.88503253796095

True Positive =  58
True Nigative=  75
False Positive =  17
False Nigative=  25
Accuracy on STUDY level = 76.0


Results for  XR_FOREARM
Total number of images =  301
True Positive =  77
True Nigative=  137
False Positive =  13
False Nigative=  74
Accuracy on IMAGE level = 71.09634551495017

True Positive =  35
True Nigative=  66
False Positive =  3
False Nigative=  29
Accuracy on STUDY level = 75.93984962406014


Results for  XR_HAND
Total number of images =  460
True Positive =  103
True Nigative=  224
False